In [1]:
#importacion de libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import FeatureHasher

import warnings
warnings.filterwarnings('ignore')

In [2]:
#carga de datasets
#Le especifico el tipo a algunas columnas porque sino me tira un warning, podria especificar el verdadero tipo tambien
todoSet = pd.read_csv("events_up_to_01062018.csv",\
                   dtype = {3: object,5: object,
                    6: object,7: object,
                    8: object,9: object,
                    10: object,11: object,
                    12: object,13: object,
                    14: object,15: object,
                    16: object,17: object,
                    18: object,19: object,
                    20: object,21: object,
                    22: object})
usersJunioSet = pd.read_csv("trocafone_kaggle_test.csv")

In [3]:
usersJunioSet.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [4]:
todoSet[todoSet['person']=='40bf23ab']

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
2341644,2018-05-30 12:22:07,lead,40bf23ab,NaN,NaN,iPhone 6S,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Merge
usersTest = todoSet[todoSet['person'].isin(usersJunioSet['person'])]

In [6]:
usersTest['timestamp'] = pd.to_datetime(usersTest['timestamp'])
#Creo una columna con solo fecha sin tiempo
usersTest['fecha'] = usersTest['timestamp'].dt.date

In [7]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# FEATURES CATEGORICOS

In [8]:
#conversion de Object a datetime
todoSet['timestamp'] = pd.to_datetime(todoSet['timestamp'])
usersTest['fecha'] = pd.to_datetime(usersTest['fecha'])
usersTest['meses'] = usersTest['timestamp'].dt.month

In [9]:
usersTest['fecha'].head()

0   2018-05-18
2   2018-05-18
3   2018-05-18
4   2018-05-18
5   2018-05-18
Name: fecha, dtype: datetime64[ns]

In [10]:
usersTest['person'].nunique()

19415

In [11]:
#GENERO FEATURE SEPARADOS POR MES

def acomodar_dataframe(dataframe):
    dataframe=dataframe.pivot_table(index='person',columns='meses').reset_index()
    #dataframe.replace([np.inf, -np.inf], 0,inplace=True)
    dataframe.fillna(0, inplace=True)
    columns = dataframe.columns.tolist()
    columns_name=[]
    for x in columns:
        columns_name=columns_name+[str(x[0])+'-'+str(x[1])]
    columns_name[0]='person'
    #columns_name[1]='label'
    dataframe.columns=columns_name
    dataframe.columns = dataframe.columns.str.replace('-1', "-Enero")
    dataframe.columns = dataframe.columns.str.replace('-2', "-Febrero")
    dataframe.columns = dataframe.columns.str.replace('-3', "-Marzo")
    dataframe.columns = dataframe.columns.str.replace('-4', "-Abril")
    dataframe.columns = dataframe.columns.str.replace('-5', "-Mayo")
#DROPEANDO COLUMNAS NULAS
    #    dataframe= dataframe.loc[:, (dataframe != 0).any(axis=0)]
    return dataframe


In [12]:
def armar_feature2(feature,nombre_columna,numero_de_columnas):
    
    #GENERO PARAMETRO FEATURE HASHING
    col=[]
    columna=nombre_columna
    cant_elem=numero_de_columnas
    #GENERO NOMBRE DE LAS COLUMNAS
    for x in range(1,cant_elem+1):
        col.append("F13_"+columna+"_fh"+str(x))
    #APLICO FEATURE HASHING Y CONCATENO
    fh = FeatureHasher(n_features=cant_elem, input_type='string')
    sp = fh.fit_transform(feature[columna])
    sp=sp.toarray().astype('int8')
    df = pd.DataFrame(sp, columns=col)
    df.reset_index(drop=True, inplace=True)
    feature.reset_index(drop=True, inplace=True)
    #TODA LA DATA JUNTA
    feature=pd.concat([feature, df], axis=1)    
    
    return [feature,cant_elem]

In [13]:
#RECUPERO Y REARMO EL DICCIONARIO

data_ref=pd.read_csv('dict_with_feature_and_number_of_columns.csv')
data_ref=data_ref[['feature','columns']]
data_ref=data_ref.set_index('feature')
data_ref=data_ref.to_dict()['columns']

data_ref

{'usersCampaignSource': 24L,
 'usersCity': 1855L,
 'usersColors': 64L,
 'usersColorsCheckouts': 52L,
 'usersColorsConversions': 35L,
 'usersConditionCheckouts': 6L,
 'usersConditionConversions': 6L,
 'usersConditionViewed': 6L,
 'usersCountry': 40L,
 'usersDevicesTypes': 5L,
 'usersModelosVistos': 194L,
 'usersModelsCheckouts': 169L,
 'usersModelsConversions': 138L,
 'usersRegionAccess': 85L,
 'usersScreenResolution': 286L,
 'usersSearchTerms': 80L,
 'usersSkusViewedBrandListing': 137L,
 'usersSkusViewedHomePage': 58L,
 'usersSkusViewedInSearchedProducts': 111L,
 'usersStorageConversion': 9L}

In [14]:
usersTest

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,fecha,meses
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
5,2018-05-18 00:44:27,searched products,4c8a8b93,NaN,NaN,NaN,NaN,NaN,NaN,"10240,9987,10322,10085,9944,9931,13404,10154,1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
7,2018-05-18 00:44:02,viewed product,29ebb414,NaN,2845.0,iPhone 6 Plus,Bom,128GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
11,2018-05-18 00:41:00,viewed product,3dc1950f,NaN,10407.0,Motorola Moto G4 Plus,Muito Bom,32GB,Preto Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
12,2018-05-18 00:40:26,viewed product,3dc1950f,NaN,3983.0,iPad Air Wi-Fi,Bom,16GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
13,2018-05-18 00:40:14,viewed product,3dc1950f,NaN,3983.0,iPad Air Wi-Fi,Bom,16GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5
15,2018-05-18 00:37:51,viewed product,8ea4c165,NaN,3347.0,Samsung Galaxy S6 Flat,Bom,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18,5


In [15]:
#13mo feature: colores vistos por el usuario por dia
usersColors = usersTest[usersTest['event'] == 'viewed product']\
              [['fecha','meses','person','color']].drop_duplicates()

usersColors=armar_feature2(usersColors,'color',data_ref['usersColors'])
usersColors=acomodar_dataframe(usersColors[0])
usersColors.shape

(18560, 321)

In [16]:
#14mo feature: colores comprados por el usuario por dia
usersColorsConversions = usersTest[usersTest['event'] == 'conversion']\
                         [['fecha','meses','person','color']].drop_duplicates()

usersColorsConversions=armar_feature2(usersColorsConversions,'color',data_ref['usersColorsConversions'])
usersColorsConversions=acomodar_dataframe(usersColorsConversions[0])
usersColorsConversions.shape

(2129, 176)

In [17]:
#15mo feature: colores que llegan al checkout por usuario por dia
usersColorsCheckouts = usersTest[usersTest['event'] == 'checkout']\
                         [['fecha','meses','person','color']].drop_duplicates()

usersColorsCheckouts=armar_feature2(usersColorsCheckouts,'color',data_ref['usersColorsCheckouts'])
usersColorsCheckouts=acomodar_dataframe(usersColorsCheckouts[0])
usersColorsCheckouts.shape

(16381, 261)

In [18]:
#16mo feature: condition de modelos con checkouts por usuario por dia
usersConditionCheckouts = usersTest[usersTest['event'] == 'checkout']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionCheckouts=armar_feature2(usersConditionCheckouts,'condition',data_ref['usersConditionCheckouts'])
usersConditionCheckouts=acomodar_dataframe(usersConditionCheckouts[0])
usersConditionCheckouts.shape

(16381, 31)

In [19]:
#17mo feature: condition de modelos con conversiones por usuario por dia
usersConditionConversions = usersTest[usersTest['event'] == 'conversion']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionConversions=armar_feature2(usersConditionConversions,'condition',data_ref['usersConditionConversions'])
usersConditionConversions=acomodar_dataframe(usersConditionConversions[0])
usersConditionConversions.shape

(2129, 31)

In [20]:
#18mo feature: condition de modelos vistos por usuario por dia
usersConditionViewed = usersTest[usersTest['event'] == 'viewed product']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionViewed=armar_feature2(usersConditionViewed,'condition',data_ref['usersConditionViewed'])
usersConditionViewed=acomodar_dataframe(usersConditionViewed[0])
usersConditionViewed.shape

(18560, 31)

In [21]:
#19mo feature: modelo visto por usuario por dia 
usersModelosVistos = usersTest[usersTest['event'] == 'viewed product']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelosVistos=armar_feature2(usersModelosVistos,'model',data_ref['usersModelosVistos'])
usersModelosVistos=acomodar_dataframe(usersModelosVistos[0])
usersModelosVistos.shape

(18560, 971)

In [22]:
#20mo feature: modelo con checkouts por usuario por dia
usersModelsCheckouts = usersTest[usersTest['event'] == 'checkout']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelsCheckouts=armar_feature2(usersModelsCheckouts,'model',data_ref['usersModelsCheckouts'])
usersModelsCheckouts=acomodar_dataframe(usersModelsCheckouts[0])
usersModelsCheckouts.shape

(16381, 846)

In [23]:
#21mo feature: modelo con conversiones por usuario por dia
usersModelsConversions = usersTest[usersTest['event'] == 'conversion']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelsConversions=armar_feature2(usersModelsConversions,'model',data_ref['usersModelsConversions'])
usersModelsConversions=acomodar_dataframe(usersModelsConversions[0])
usersModelsConversions.shape

(2129, 691)

In [24]:
#22mo feature: skus vistos en cierta busqueda en el site por usuario por dia
usersSkusViewedInSearchedProducts = usersTest[usersTest['event'] == 'searched products']\
                 [['fecha','meses','person','skus']].dropna().drop_duplicates()

usersSkusViewedInSearchedProducts=armar_feature2(usersSkusViewedInSearchedProducts,'skus',data_ref['usersSkusViewedInSearchedProducts'])
usersSkusViewedInSearchedProducts=acomodar_dataframe(usersSkusViewedInSearchedProducts[0])
usersSkusViewedInSearchedProducts.shape

(5850, 556)

In [25]:
#23mo feature: skus vistos en la home page por user por dia
usersSkusViewedHomePage = usersTest[usersTest['event'] == 'generic listing']\
                 [['fecha','meses','person','skus']].dropna().drop_duplicates()

usersSkusViewedHomePage=armar_feature2(usersSkusViewedHomePage,'skus',data_ref['usersSkusViewedHomePage'])
usersSkusViewedHomePage=acomodar_dataframe(usersSkusViewedHomePage[0])
usersSkusViewedHomePage.shape

(12578, 291)

In [26]:
#24mo feature: skus vistos en un listado especifico de una marca por user por dia
usersSkusViewedBrandListing = usersTest[usersTest['event'] == 'brand listing']\
                              [['fecha','meses','person','skus']].dropna().drop_duplicates()


usersSkusViewedBrandListing=armar_feature2(usersSkusViewedBrandListing,'skus',data_ref['usersSkusViewedBrandListing'])
usersSkusViewedBrandListing=acomodar_dataframe(usersSkusViewedBrandListing[0])
usersSkusViewedBrandListing.shape

(9621, 686)

In [27]:
#25mo feature: search terms por usuario por dia
usersSearchTerms = usersTest[usersTest['event'] == 'searched products']\
                 [['fecha','meses','person','search_term']].dropna().drop_duplicates()

usersSearchTerms=armar_feature2(usersSearchTerms,'search_term',data_ref['usersSearchTerms'])
usersSearchTerms=acomodar_dataframe(usersSearchTerms[0])
usersSearchTerms.shape

(6297, 401)

In [28]:
#26mo feature: screen resolution por usuario por dia 
usersScreenResolution = usersTest[usersTest['event'] == 'visited site']\
                 [['fecha','meses','person','screen_resolution']].dropna().drop_duplicates()

usersScreenResolution=armar_feature2(usersScreenResolution,'screen_resolution',data_ref['usersScreenResolution'])
usersScreenResolution=acomodar_dataframe(usersScreenResolution[0])
usersScreenResolution.shape

(19115, 1431)

In [29]:
#27mo feature: storage de dispositivos convertidos por usuario por dia
usersStorageConversion = usersTest[usersTest['event'] == 'conversion']\
                 [['fecha','meses','person','storage']].dropna().drop_duplicates()

usersStorageConversion=armar_feature2(usersStorageConversion,'storage',data_ref['usersStorageConversion'])
usersStorageConversion=acomodar_dataframe(usersStorageConversion[0])
usersStorageConversion.shape

(2129, 46)

In [30]:
#28mo feature: campaign source por user por dia
usersCampaignSource =  usersTest[usersTest['event'] == 'ad campaign hit']\
                 [['fecha','meses','person','campaign_source']].dropna().drop_duplicates()

usersCampaignSource=armar_feature2(usersCampaignSource,'campaign_source',data_ref['usersCampaignSource'])
usersCampaignSource=acomodar_dataframe(usersCampaignSource[0])
usersCampaignSource.shape

(16003, 121)

In [31]:
#29mo feature: country access user
usersCountry = usersTest[usersTest['event'] == 'visited site']\
                 [['fecha','meses','person','country']].dropna().drop_duplicates()

usersCountry=armar_feature2(usersCountry,'country',data_ref['usersCountry'])
usersCountry=acomodar_dataframe(usersCountry[0])
usersCountry.shape

(19116, 201)

In [32]:
#30mo feature: city access user
usersCity = usersTest[usersTest['event'] == 'visited site']\
                 [['fecha','meses','person','city']].dropna().drop_duplicates()

usersCity=armar_feature2(usersCity,'city',data_ref['usersCity'])
usersCity=acomodar_dataframe(usersCity[0])
usersCity.shape

(19116, 9276)

In [33]:
#31mo feature: region access user
usersRegionAccess = usersTest[usersTest['event'] == 'visited site']\
                 [['fecha','meses','person','region']].dropna().drop_duplicates()


usersRegionAccess=armar_feature2(usersRegionAccess,'region',data_ref['usersRegionAccess'])
usersRegionAccess=acomodar_dataframe(usersRegionAccess[0])
usersRegionAccess.shape

(19116, 426)

In [34]:
#32mo feature: device type visited site users
usersDevicesTypes = usersTest[usersTest['event'] == 'visited site']\
                 [['fecha','meses','person','device_type']].dropna().drop_duplicates()

usersDevicesTypes=armar_feature2(usersDevicesTypes,'device_type',data_ref['usersDevicesTypes'])
usersDevicesTypes=acomodar_dataframe(usersDevicesTypes[0])
usersDevicesTypes.shape#['person'].nunique()

(19116, 26)

### COMENZAMOS CON EL JOIN DE LOS FEATURES CATEGORICOS
### PRIMERO GENERAMOS EL DF DONDE REALIZAREMOS EL MERGE

In [35]:
join_features_categoricos=usersTest.loc[:,['person']].drop_duplicates()
join_features_categoricos.head()

,person
0,4886f805
2,0297fc1e
3,2d681dd8
4,cccea85e
5,4c8a8b93


In [36]:
join_features_categoricos.shape

(19415, 1)

In [37]:
#GENERO UNA LISTA DE DF PARA REALIZAR EL MERGE
features_categoricos=[usersColors,usersColorsConversions,usersColorsCheckouts,usersConditionCheckouts,usersConditionConversions,usersConditionViewed,usersModelosVistos,usersModelsCheckouts,usersModelsConversions,usersSkusViewedInSearchedProducts,usersSkusViewedHomePage,usersSkusViewedBrandListing,usersSearchTerms,usersScreenResolution,usersStorageConversion,usersCampaignSource,usersCountry,usersCity,usersRegionAccess,usersDevicesTypes]
print ("Cantidad de features categoricos",len(features_categoricos))#32-13+1=20

('Cantidad de features categoricos', 20)


In [38]:
#HAGO EL MERGE CON EL DF CREADO CON PERSON Y LABEL
for feature_to_merge in features_categoricos:
    join_features_categoricos=pd.merge(join_features_categoricos,feature_to_merge, on=['person'],how='left')

In [39]:
#QUITO LOS NA Y COLOCO 0 Y RENOMBRO LAS COLUMNAS Y GUARDO EL ARCHIVO
print join_features_categoricos.shape
join_features_categoricos=join_features_categoricos.fillna(0)
join_features_categoricos.columns = list (join_features_categoricos.columns.str.replace('_x',''))
join_features_categoricos.columns = list (join_features_categoricos.columns.str.replace('_y',''))
join_features_categoricos.to_csv('join_features_categoricos_test_final.csv')
join_features_categoricos.head()

(19415, 16801)


,person,F13_color_fh1-Enero,F13_color_fh1-Febrero,F13_color_fh1-Marzo,F13_color_fh1-Abril,F13_color_fh1-Mayo,F13_color_fh10-Enero,F13_color_fh10-Febrero,F13_color_fh10-Marzo,F13_color_fh10-Abril,...,F13_device_type_fh4-Enero,F13_device_type_fh4-Febrero,F13_device_type_fh4-Marzo,F13_device_type_fh4-Abril,F13_device_type_fh4-Mayo,F13_device_type_fh5-Enero,F13_device_type_fh5-Febrero,F13_device_type_fh5-Marzo,F13_device_type_fh5-Abril,F13_device_type_fh5-Mayo
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1,0297fc1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,cccea85e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4c8a8b93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
